# libraries

In [2]:
import pandas as pd
import json
import numpy as np
import math


In [166]:
def compose_coordinate(latitude, longitude):
    composed_coordinate = '['+ str(longitude)+ ','+ str(latitude)+ ']'
    return composed_coordinate

# def haversine_distance(coord1, coord2):
#     R = 6371.0  # Earth's radius in km
#     lat1 = math.radians(coord1[0])
#     lon1 = math.radians(coord1[1])
#     lat2 = math.radians(coord2[0])
#     lon2 = math.radians(coord2[1])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     distance = R * c
#     return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c

    #distance = ((lat1-lat2)**2+(lon1-lon2)**2)**0.5



    return distance


# import data

In [4]:
total_node = pd.read_csv("./total_node.csv")
total_link = pd.read_csv("./total_link.csv")
print("노드의 크기:",total_node.shape)
print("링크의 크기:",total_link.shape)

노드의 크기: (461110, 2)
링크의 크기: (361526, 17)


### 노드 반올림

In [5]:
total_node['경도반올림'] = total_node['경도'].round(9)
total_node['위도반올림'] = total_node['위도'].round(9)

### 중복 노드 제거 및 링크 넘버링

In [6]:
total_node.drop_duplicates(keep='first')

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127
...,...,...,...,...
461102,127.001000,37.230355,127.001000,37.230355
461103,127.001046,37.230343,127.001046,37.230343
461107,127.025045,37.235468,127.025045,37.235468
461108,127.025000,37.235515,127.025000,37.235515


In [7]:
total_node.duplicated(['경도반올림','위도반올림']).sum()

108609

In [8]:
total_node.duplicated(['경도','위도']).sum()

108609

In [9]:
total_node.head()

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127


In [10]:
total_link.head()

,노드1,노드2,거리,경찰서,공원,cctv,가로등,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,"[126.8962667073345, 37.25877484423394]","[126.89626899673534, 37.25879828123242]",0.002614,9.941103,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[126.99492119734404, 37.25307704311938]","[126.99490426643689, 37.25311243627993]",0.004211,6.405129,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[127.00271514375935, 37.25412660967416]","[127.0023185009589, 37.254061429039]",0.035846,6.395166,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[127.0023185009589, 37.254061429039]","[127.00192790991031, 37.25400274184614]",0.035180,6.395011,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[127.00192790991031, 37.25400274184614]","[127.00142049616967, 37.25393654954797]",0.045509,6.394178,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#link의 weight을 만들었다. 나중에 거리와 위험도를 종합적으로 따져 weight값을 정할 것이고 현재는 그냥 거리값만 적용시켰다.
total_link.insert(2, 'weight', total_link['거리'].copy())



### 합쳐진 경도 위도 나누기

In [12]:
templink = pd.DataFrame( columns=['노드1경도', '노드1위도','노드2경도','노드2위도'])
for i in range(total_link.shape[0]):
    nodedata = json.loads(total_link['노드1'][i])
    nodedata.extend(json.loads(total_link['노드2'][i]))
    templink.loc[i] = nodedata

templink.head()


,노드1경도,노드1위도,노드2경도,노드2위도
0,126.896267,37.258775,126.896269,37.258798
1,126.994921,37.253077,126.994904,37.253112
2,127.002715,37.254127,127.002319,37.254061
3,127.002319,37.254061,127.001928,37.254003
4,127.001928,37.254003,127.001420,37.253937


### 반올림

In [13]:
#optional

templink['노드1경도'] = templink['노드1경도'].round(9)
templink['노드1위도'] = templink['노드1위도'].round(9)
templink['노드2경도'] = templink['노드2경도'].round(9)
templink['노드2위도'] = templink['노드2위도'].round(9)

### 노드 인덱스 연결

In [14]:
#optional

node1connect = []
node2connect = []

for i in range(templink.shape[0]):
    
    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드1위도'][i]) & (total_node['경도반올림'] == templink['노드1경도'][i])]
    try:
        node1connect.append(contemp.index[0])
    except:
        print("{}번째 데이터 [{},{}] not in node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))
        node1connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))

    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드2위도'][i]) & (total_node['경도반올림'] == templink['노드2경도'][i])]
    try:
        node2connect.append(contemp.index[0])
    except:
        print("{}번째 데이터[{},{}] not in node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))
        node2connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))

templink.insert(0, 'idx노드1', node1connect)
templink.insert(1, 'idx노드2', node2connect)


80273번째 데이터[37.326324665,126.957343228] not in node
80274번째 데이터 [37.326324665,126.957343228] not in node
133407번째 데이터[37.218289837,126.964691178] not in node
133408번째 데이터 [37.218289837,126.964691178] not in node


## 임시로 위 2개 데이터 삭제할것

In [15]:
total_link = pd.concat([templink, total_link], axis=1)

total_link = total_link.drop([80273, 80274, 133407, 133408])


total_link.head()


,idx노드1,idx노드2,노드1경도,노드1위도,노드2경도,노드2위도,노드1,노드2,weight,거리,...,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,0.0,1.0,126.896267,37.258775,126.896269,37.258798,"[126.8962667073345, 37.25877484423394]","[126.89626899673534, 37.25879828123242]",0.002614,0.002614,...,0,0,0,0,0,0,0,0,0,0
1,2.0,3.0,126.994921,37.253077,126.994904,37.253112,"[126.99492119734404, 37.25307704311938]","[126.99490426643689, 37.25311243627993]",0.004211,0.004211,...,0,0,0,0,0,0,0,0,0,0
2,4.0,5.0,127.002715,37.254127,127.002319,37.254061,"[127.00271514375935, 37.25412660967416]","[127.0023185009589, 37.254061429039]",0.035846,0.035846,...,0,0,0,0,0,0,0,0,0,0
3,5.0,6.0,127.002319,37.254061,127.001928,37.254003,"[127.0023185009589, 37.254061429039]","[127.00192790991031, 37.25400274184614]",0.035180,0.035180,...,0,0,0,0,0,0,0,0,0,0
4,6.0,7.0,127.001928,37.254003,127.001420,37.253937,"[127.00192790991031, 37.25400274184614]","[127.00142049616967, 37.25393654954797]",0.045509,0.045509,...,0,0,0,0,0,0,0,0,0,0


In [16]:
total_link.to_csv("D:/modified_link.csv", index=True)

# 시작점 입력

In [306]:
dep_lat, dep_lon = 37.294195879066635, 127.0033176194018
des_lat, des_lon = 37.29475413961952, 127.00458455362698

# dep_lat = 37.28293695757347
# dep_lon = 127.0434230405271
# des_lat = 37.28312042096528
# des_lon = 127.04739397575078

# Get Nearest Node
출발점과 도착점의 가장 가까운 노드를 찾는 기능이다. 

In [307]:
def get_Nearest_Node(latitude, longitude):
    length_filter = 0.0003
    min_distance = 1e9
    nearest_lat = 0
    nearest_lon = 0
    nearest_idx = 0
    
    filtered_df = total_node.loc[(total_node['위도'] > latitude - length_filter) & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] > longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]
    for index, row in filtered_df.iterrows():

        cur_node_lat = row['위도']
        cur_node_lon = row['경도']
        cur_distance = haversine_distance(latitude, longitude, cur_node_lat, cur_node_lon)

        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_lat = cur_node_lat
            nearest_lon = cur_node_lon
            nearest_idx = index

    #return filtered_df
    return nearest_idx, nearest_lat, nearest_lon
    

start_idx, start_lat, start_lon = get_Nearest_Node(dep_lat, dep_lon)
end_idx, end_lat, end_lon = get_Nearest_Node(des_lat, des_lon)


In [308]:
print(start_idx, end_idx)

416241 407660


# 다익스트라 알고리즘

In [309]:
area_N = max(dep_lat, des_lat) + 0.005
area_S = min(dep_lat, des_lat) - 0.005
area_E = max(dep_lon, des_lon) + 0.005
area_W = min(dep_lon, des_lon) - 0.005

areal_link = total_link.loc[(total_link['노드1위도'] < area_N)
                             & (total_link['노드2위도'] < area_N)
                             & (total_link['노드1위도'] > area_S) 
                             & (total_link['노드2위도'] > area_S)
                             & (total_link['노드1경도'] < area_E)
                             & (total_link['노드2경도'] < area_E)
                             & (total_link['노드1경도'] > area_W)
                             & (total_link['노드2경도'] > area_W)]

In [310]:
areal_link

,idx노드1,idx노드2,노드1경도,노드1위도,노드2경도,노드2위도,노드1,노드2,weight,거리,...,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
32466,36079.0,36080.0,126.998672,37.290326,126.998629,37.290308,"[126.99867183768276, 37.29032589548867]","[126.99862884369283, 37.29030784243086]",0.004301,0.004301,...,0,0,0,0,0,0,0,0,0,0
32467,36080.0,36081.0,126.998629,37.290308,126.998370,37.290195,"[126.99862884369283, 37.29030784243086]","[126.99837036428146, 37.29019515433979]",0.026074,0.026074,...,0,0,0,0,0,0,0,0,0,0
32469,36083.0,36079.0,126.998745,37.290357,126.998672,37.290326,"[126.99874520371186, 37.29035670201854]","[126.99867183768276, 37.29032589548867]",0.007339,0.007339,...,0,0,0,0,0,0,0,0,0,0
44483,51641.0,51642.0,127.008085,37.289201,127.008061,37.289210,"[127.00808524170385, 37.2892009648496]","[127.0080614867841, 37.28920998999186]",0.002329,0.002329,...,0,0,1,0,0,0,0,0,0,1
44484,51642.0,51643.0,127.008061,37.289210,127.008059,37.289211,"[127.0080614867841, 37.28920998999186]","[127.00805875819219, 37.28921130549739]",0.000282,0.000282,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335144,428377.0,356231.0,127.004900,37.298167,127.005061,37.298351,"[127.00490005569006, 37.298167319771686]","[127.00506082174694, 37.29835083703431]",0.024872,0.024872,...,0,0,0,0,0,5,0,0,0,0
335145,407984.0,428380.0,127.003824,37.297903,127.003882,37.297968,"[127.00382412382375, 37.29790251825934]","[127.00388192986635, 37.297968363640166]",0.008930,0.008930,...,0,0,0,0,0,9,0,0,0,0
335146,428380.0,357045.0,127.003882,37.297968,127.003939,37.298034,"[127.00388192986635, 37.297968363640166]","[127.00393930813406, 37.298033731266244]",0.008865,0.008865,...,0,0,0,0,0,7,0,0,0,0
335147,357076.0,428383.0,127.003526,37.298034,127.003624,37.297978,"[127.00352610359054, 37.29803423230363]","[127.00362364946791, 37.29797801262323]",0.010655,0.010655,...,0,0,0,0,0,9,0,0,0,0


In [311]:
num_rows = total_node.shape[0]
dijkstra_df = {'col1': [np.inf] * num_rows, 'col2': [False] * num_rows, 'col3': [np.nan] * num_rows}
dijkstra_df = pd.DataFrame(data=dijkstra_df)
dijkstra_df.columns=['length', 'visited', 'route']

In [312]:
# start_idx = 33568
# #start_idx = 19265
# end_idx = 4

In [313]:
dijkstra_df.loc[start_idx, "length"] = 0
dijkstra_df.at[start_idx, "route"] = start_idx
now = start_idx

#num_visited = 0
while(True):

    unvisited_dijkstra_df = dijkstra_df.loc[(dijkstra_df['visited']  == False) & (dijkstra_df['length']  != np.inf)]
    if unvisited_dijkstra_df.empty:
        print('unvisited empty')
        break

    unvisited_dijkstra_df.sort_values(by=['length'])
    now = unvisited_dijkstra_df.index[0]
    #num_visited += 1
    #print("visited", now, "total visit:", num_visited)
    now_length = dijkstra_df.loc[now, "length"]########################startidx 가 아니라 now 아닌가?????
    
    dijkstra_df.loc[now, "visited"] = True
    #dijkstra_df.loc[now, "route"] = [start_idx]

    now_connected_link = areal_link.loc[(areal_link['idx노드1']  == now)]
    for index, row in now_connected_link.iterrows():
        search_node_num = row['idx노드2']
        search_node_weight = row['weight']
        #print(now, ' is connected to ', search_node_num)

        if dijkstra_df.loc[search_node_num, "visited"] == True:
            continue
        
        if dijkstra_df.loc[search_node_num, "length"] > (now_length + search_node_weight):
            dijkstra_df.loc[search_node_num, "length"] = now_length + search_node_weight
            # templist = dijkstra_df.loc[now, "route"].copy()
            # templist.append(now)
            # print(templist)
            dijkstra_df.at[search_node_num, "route"] = now
            #print("modified length of ",search_node_num)


    now_connected_link = areal_link.loc[(areal_link['idx노드2']  == now)]
    for index, row in now_connected_link.iterrows():
        search_node_num = row['idx노드1']
        search_node_weight = row['weight']
        #print(now, ' is connected to ', search_node_num)

        if dijkstra_df.loc[search_node_num, "visited"] == True:
            continue
        
        if dijkstra_df.loc[search_node_num, "length"] > (now_length + search_node_weight):
            dijkstra_df.loc[search_node_num, "length"] = now_length + search_node_weight
            # templist = dijkstra_df.loc[now, "route"].copy()
            # templist.append(now)
            # print(templist)
            dijkstra_df.at[search_node_num, "route"] = now
            #print("modified length of ",search_node_num)


unvisited empty


# 최적화 방법

특정 사각형 내에서만 link를 제한하여 그 안에서만 경로를 돌린다

dictionary등으로 변경해본다

첫번째 경로 찾으면 끊는다? 이게 과연 최단일까?

# 결과 및 경로

In [314]:
route_list = [[des_lat,des_lon]]
cur_idx = end_idx


while(True):
    
    cur_lat = total_node.loc[cur_idx, "위도"]
    cur_lon = total_node.loc[cur_idx, "경도"]
    route_list.append([cur_lat, cur_lon])
    if cur_idx == start_idx:
        break
    
    cur_idx = dijkstra_df.loc[cur_idx, "route"]

route_list.append([dep_lat, dep_lon])


In [315]:
route_list

[[37.29475413961952, 127.00458455362698],
 [37.29476625995413, 127.00458054988128],
 [37.29484532353671, 127.00485692617244],
 [37.294924386524215, 127.00513331433076],
 [37.29492890900437, 127.00514910289844],
 [37.29491499367322, 127.00522507171982],
 [37.29487378839151, 127.00525502381204],
 [37.29444801630903, 127.00556454553902],
 [37.29431093643705, 127.0056641893951],
 [37.29402384615747, 127.00587754858071],
 [37.2940017621124, 127.00589395555204],
 [37.29398077086101, 127.00583486900636],
 [37.29389463304408, 127.00559244034336],
 [37.293808505691146, 127.00535004601323],
 [37.29377965720809, 127.00526885480642],
 [37.29375025062694, 127.00520338372164],
 [37.29369496946792, 127.00508031399468],
 [37.29363969543983, 127.00495725568366],
 [37.29363288419599, 127.00494208713076],
 [37.29362560374992, 127.00492680431842],
 [37.293569698792645, 127.00480939607552],
 [37.293513795626254, 127.00469201281598],
 [37.29350465993383, 127.00467281594788],
 [37.29349001862119, 127.0046420

# 아래는 안쓰는 코드

In [316]:
now_connected_link = total_link.loc[(total_link['idx노드1']  == now)]
for i in range(now_connected_link.shape[0]):
    

SyntaxError: incomplete input (2211525418.py, line 3)

In [ ]:
#get_Nearest_Node 함수가 완성되어 가장 가까운 노드가 여기라고 가정하자

start_lat = 37.2793947321759
start_lon = 127.016818947031
end_lat = 37.2795395918437
end_lon = 127.016815561923



#이거말고 start idx end idx 해야 편할듯

In [ ]:
tempstart_df = total_link.loc[(total_link['노드1']  == start_coordinate)]# & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] < longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]


tempstart_df

,노드1,노드2,weight,거리,경찰서,공원,cctv,가로등,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설


In [ ]:


filnode_lat = total_node['위도'][458109]
filnode_lon = total_node['경도'][458109]


filnode_df = total_node.loc[(total_node['위도'] == filnode_lat) & (total_node['경도'] == filnode_lon)]
filnode_df

fillink_df

,경도,위도
458108,127.039858,37.237389
458109,127.039858,37.237389


In [ ]:
#링크와 노드 데이터가 매칭되는지 확인했던 함수이며 안쓰일것


for i in range(461109):
    filnode_lat = total_node['위도반올림'][i]
    filnode_lon = total_node['경도반올림'][i]
    
    fillink_df = total_link.loc[(total_link['노드1위도']  == filnode_lat) & (total_link['노드1경도']  == filnode_lon)]
    if fillink_df.shape[0] > 1:
        print(fillink_df.shape, i)
    if i % 10000 == 0:
        print(i)

0
(2, 22) 799
(2, 22) 2157
(2, 22) 2500
(2, 22) 6696
(3, 22) 7321
(2, 22) 9026
(2, 22) 9304
10000
(2, 22) 19265
(2, 22) 19290
(2, 22) 19408
20000
(2, 22) 23065
(2, 22) 25223
(2, 22) 25795
(2, 22) 29056
(2, 22) 29963
30000
(2, 22) 31996
(2, 22) 33087
(2, 22) 33095
(2, 22) 33126
(2, 22) 33320
(2, 22) 33568
(2, 22) 33849
(2, 22) 33857
(2, 22) 33979
(2, 22) 34039
(2, 22) 34970
(2, 22) 35015
(2, 22) 35081
(2, 22) 35157
(4, 22) 35707
(3, 22) 35997
(2, 22) 36233
(3, 22) 36269
40000
(2, 22) 40006
(2, 22) 40955
(2, 22) 45520
(2, 22) 47462
(2, 22) 48495
50000
(2, 22) 50713
(2, 22) 50720
(2, 22) 50740
(2, 22) 50790


KeyboardInterrupt: 

In [ ]:
for i in range(461109):
    filnode_lat = total_node['위도'][i]
    filnode_lon = total_node['경도'][i]
    

127.016819
